In [131]:
import pandas as pd
import os
import linecache
import datetime
from datetime import datetime as dt

In [132]:
file = '570342027_AnMed Cannon_standardcharges (3).xlsx'
actual_file = ".\\input_files\\Absolute Total Care Medicaid.csv"

In [133]:
date_updated = linecache.getline(actual_file, 2).split(": ")[-1].split(",")[0]
date_updated = dt.strptime(date_updated, '%m/%d/%Y').strftime('%Y-%m-%d')


In [134]:
df = pd.read_csv(actual_file, skiprows=5)

In [135]:
df.rename(columns = {
    'ContractName': 'payer_name',
    'CodeType': 'code_type',
    'Code': 'code',
    'Description': 'description',
    'Service Description': 'rev_desc',
    'Base Rate Methodology': 'rev_method',
    'ServiceCode': 'patient_class',
    'Negotiated Charge / Base Rate': 'rate'
}, inplace = True)

In [136]:
# df['payer_orig'] = df['payer_name']
df['code_orig'] = df['code']

In [137]:
df['code_type'] = df['code_type'].str.lower()

In [138]:
df['patient_class'] = df['patient_class'].str.replace('IP', 'inpatient').str.replace('OP', 'outpatient')
# Set rev_code equal to code and set code_type to None if code_type equals revenuecode
df.loc[df["code_type"] == "revenuecode", "rev_code"] = df["code"]

In [139]:
df.loc[df["code_type"] == "revenuecode"]

,payer_name,code_type,code,description,rev_desc,rev_method,rate,patient_class,Deidentified Minimum Rate,Deidentified Maximum Rate,code_orig,rev_code
1204,Absolute Total Care Medicaid,revenuecode,0170,Nursery,Nursery,Per Unit,$379.80,outpatient,379.80,379.80,0170,0170
1205,Absolute Total Care Medicaid,revenuecode,0171,Newborn-Level I,Nursery,Per Unit,$379.80,outpatient,379.80,379.80,0171,0171
1206,Absolute Total Care Medicaid,revenuecode,0258,Pharmacy: IV solutions,IV Solutions,Per Unit,$47.94,outpatient,47.94,47.94,0258,0258
1207,Absolute Total Care Medicaid,revenuecode,0260,IV Therapy,IV Therapy,Per Unit,$46.09,outpatient,46.09,46.09,0260,0260
1208,Absolute Total Care Medicaid,revenuecode,0261,IV Therapy: Infusion pump,IV Therapy,Per Unit,$46.09,outpatient,46.09,46.09,0261,0261
...,...,...,...,...,...,...,...,...,...,...,...,...
72667,Absolute Total Care Medicaid,revenuecode,0921-70010-88749,NaN,Peripheral Vascular,Per Unit,$355.83,outpatient,355.83,355.83,0921-70010-88749,0921-70010-88749
72668,Absolute Total Care Medicaid,revenuecode,0922-70010-88749,NaN,Electromyelogram,Per Unit,$68.22,outpatient,68.22,68.22,0922-70010-88749,0922-70010-88749
72669,Absolute Total Care Medicaid,revenuecode,0924-70010-88749,NaN,Allergy Test,Per Unit,$55.31,outpatient,55.31,55.31,0924-70010-88749,0924-70010-88749
72670,Absolute Total Care Medicaid,revenuecode,0940 90281-99607,NaN,Other Therapeutic Services,Per Unit,$90.35,outpatient,90.35,90.35,0940 90281-99607,0940 90281-99607


In [140]:
df1 = df.drop(['payer_name', 'rate'], axis=1)
cols = df1.columns.tolist()
value_vars = ['Deidentified Minimum Rate', 'Deidentified Maximum Rate']
id_vars = [x for x in cols if x not in value_vars]

df1 = pd.melt(df1, id_vars=id_vars, value_vars=value_vars, var_name='payer_name', value_name='rate')


In [141]:
df1.loc[df1['code'] == 'U0001']


,code_type,code,description,rev_desc,rev_method,patient_class,code_orig,rev_code,payer_name,rate
73660,cpt,U0001,2019-ncov diagnostic p,OP FS 20200315,Per Unit,outpatient,U0001,NaN,Deidentified Minimum Rate,28.18
147324,cpt,U0001,2019-ncov diagnostic p,OP FS 20200315,Per Unit,outpatient,U0001,NaN,Deidentified Maximum Rate,45.62


In [143]:
df.loc[df['code'] == 'U0001']
df.drop(value_vars, axis=1, inplace=True)

Index(['payer_name', 'code_type', 'code', 'description', 'rev_desc',
       'rev_method', 'rate', 'patient_class', 'code_orig', 'rev_code'],
      dtype='object')

In [144]:
df_cols = df.columns.values.tolist()
df_cols.sort()

df  = df[df_cols]
df1 = df1[df_cols]

In [148]:
df3 = df.append(df1)

C:\Users\adria\AppData\Local\Temp\ipykernel_23360\1388191075.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df.append(df1)


In [150]:
df3['rate'] = df3['rate'].str.replace('$', '').str.replace(',', '')

C:\Users\adria\AppData\Local\Temp\ipykernel_23360\3939721267.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df3['rate'] = df3['rate'].str.replace('$', '').str.replace(',', '')


In [151]:
df3

,code,code_orig,code_type,description,patient_class,payer_name,rate,rev_code,rev_desc,rev_method
0,0011,0011,drg,LIVER TRANSPLANT &/OR INTESTINAL TRANSPLANT,inpatient,Absolute Total Care Medicaid,68528.05,NaN,Medical/Surgical,Relative Weights
1,0012,0012,drg,LIVER TRANSPLANT &/OR INTESTINAL TRANSPLANT,inpatient,Absolute Total Care Medicaid,68910.80,NaN,Medical/Surgical,Relative Weights
2,0013,0013,drg,LIVER TRANSPLANT &/OR INTESTINAL TRANSPLANT,inpatient,Absolute Total Care Medicaid,80941.83,NaN,Medical/Surgical,Relative Weights
3,0014,0014,drg,LIVER TRANSPLANT &/OR INTESTINAL TRANSPLANT,inpatient,Absolute Total Care Medicaid,140764.58,NaN,Medical/Surgical,Relative Weights
4,0021,0021,drg,HEART &/OR LUNG TRANSPLANT,inpatient,Absolute Total Care Medicaid,74720.75,NaN,Medical/Surgical,Relative Weights
...,...,...,...,...,...,...,...,...,...,...
147323,T1001,T1001,cpt,Nursing assessment/evaluatn,outpatient,Deidentified Maximum Rate,138.13,NaN,OP FS 20190701,Per Unit
147324,U0001,U0001,cpt,2019-ncov diagnostic p,outpatient,Deidentified Maximum Rate,45.62,NaN,OP FS 20200315,Per Unit
147325,U0002,U0002,cpt,Covid-19 lab test non-cdc,outpatient,Deidentified Maximum Rate,65.18,NaN,OP FS 20200315,Per Unit
147326,U0003,U0003,cpt,Cov-19 amp prb hgh thruput,outpatient,Deidentified Maximum Rate,127.03,NaN,Clinical Lab,Per Unit
